# **Importar Datos**

In [1]:
import numpy as np
import pandas as pd
import os  ### para ver y cambiar directorio de trabajo
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import CountVectorizer
#from ipywidgets import interact
#import ipywidgets as interact
#from IPython.display import display
#from surprise import Reader, Dataset
#from surprise.model_selection import cross_validate, GridSearchCV
#from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
#from surprise.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#os.getcwd() ## ver directorio actual
#os.chdir('/content') ### cambiar directorio a ruta específica

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##### conectarse a BD #######

conn = sql.connect('/content/drive/MyDrive/Analitica III/MARKETING/db_movies (1)')
#conn=sql.connect('/content/drive/MyDrive/db_movies')
cur=conn.cursor()



In [4]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cur.fetchall())

[('ratings',), ('movies',), ('usuarios_sel',), ('peliculas_sel',), ('ratings_final',), ('peliculas_final',), ('full_ratings',), ('df',)]


In [5]:
#######
############ traer tabla de BD a python ####

movies= pd.read_sql("""select *  from movies""", conn)
ratings = pd.read_sql('select * from ratings', conn)

# **Exploración Inicial**

## **Movies**

In [6]:
### Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados
movies.info()
##No se tienen Nulos, tienen los mismo registros


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
movies.duplicated().sum() 

0

In [8]:
def dbmovies():
  movies=pd.read_sql("""select * from movies """, conn)
  genres=movies['genres'].str.split('|')
  te = TransactionEncoder()
  genres = te.fit_transform(genres)
  genres = pd.DataFrame(genres, columns = te.columns_)
  movies
  return movies
dbmovies() 

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


## **Ratings**

In [9]:
### Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados
ratings.info()
##No se tienen Nulos, tienen los mismos registros en todos las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [10]:
ratings.duplicated().sum() 

0

In [11]:
def dbratings():
  ratings=pd.read_sql("""select * from ratings """, conn)
  return ratings
dbratings()  

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


# **Descripción base**

In [12]:
###Calcular la distribución de calificaciones
cr=pd.read_sql(""" select rating,
                          count(*) as conteo 
                          from ratings
                          group by rating
                          order by conteo desc""", conn)
###Nombres de columnas con numeros o guiones se deben poner en doble comilla para que se reconozcan
###Los que estan en 0 se consideran leidos pero no calificados
#### Se conoce como calificación implicita, consume producto pero no da una calificacion
cr = cr.sort_values(by='conteo',ascending=False)

In [13]:
# Crear la figura de barras
data = go.Bar(x=cr.rating, y=cr.conteo, text=cr.conteo, textposition="outside")
layout = go.Layout(title="Count of ratings", xaxis={'title':'Rating'}, yaxis={'title':'Count'})
fig = go.Figure(data=data, layout=layout)

# Mostrar la figura
fig.show()

In [14]:
### calcular cada usuario cuátos libros calificó
rating_users=pd.read_sql(''' select userId,
                         count(*) as cnt_rat
                         from ratings
                         group by userId
                         order by "cnt_rat" asc
                         ''',conn )

fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show()

#### Se ve la mayor parte de usuarios calificando entre 0 y 300 peliculas

In [15]:
#### filtrar usuarios con más de 50 libros calificados (para tener calificaion confiable) y los que tienen menos de mill porque pueden ser no razonables
rating_users2=pd.read_sql(''' select userId,
                         count(*) as cnt_rat
                         from ratings
                         group by userId
                         having cnt_rat <=1000
                         order by cnt_rat asc
                         ''',conn )
rating_users2

,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
593,177,904
594,298,939
595,603,943
596,307,975


In [16]:
### ver distribucion despues de filtros,ahora se ve mas razonables
rating_users2.describe()

,userId,cnt_rat
count,598.000000,598.000000
mean,303.608696,137.657191
std,176.083249,169.447554
min,1.000000,20.000000
25%,151.250000,35.000000
50%,303.500000,69.000000
75%,456.750000,162.500000
max,609.000000,977.000000


In [17]:
### graficar distribucion despues de filtrar datos
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show()

In [18]:
#### verificar cuantas calificaciones tiene cada pelicula
rating_movies=pd.read_sql(''' select movieId,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         order by cnt_rat desc
                         ''',conn )

rating_movies

,movieId,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
9719,96,1
9720,83,1
9721,77,1
9722,55,1


In [19]:
### graficar distribucion

fig  = px.histogram(rating_movies, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada película')
fig.show() 

In [20]:
####Filtrar películas que tengan más de 20 calificaciones 
rating_movies2=pd.read_sql(''' select movieId,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         having cnt_rat>=20
                         order by cnt_rat desc
                         ''',conn )
rating_movies2.describe()

,movieId,cnt_rat
count,1297.000000,1297.000000
mean,16999.244410,52.350039
std,31279.458955,40.207459
min,1.000000,20.000000
25%,1221.000000,27.000000
50%,2761.000000,39.000000
75%,6957.000000,61.000000
max,168252.000000,329.000000


In [21]:
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada película')
fig.show()